In [1]:
#download das bases - arquivo completo
!wget -p https://www.anatel.gov.br/dadosabertos/paineis_de_dados/acessos/acessos_telefonia_movel.zip

--2021-10-24 10:20:22--  https://www.anatel.gov.br/dadosabertos/paineis_de_dados/acessos/acessos_telefonia_movel.zip
Resolving www.anatel.gov.br (www.anatel.gov.br)... 200.0.81.81, 2801:80:c90:c1da:da0:2312:2000:1331
Connecting to www.anatel.gov.br (www.anatel.gov.br)|200.0.81.81|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 802894421 (766M) [application/x-zip-compressed]
Saving to: ‘www.anatel.gov.br/dadosabertos/paineis_de_dados/acessos/acessos_telefonia_movel.zip’

www.anatel.gov.br/d 100%[===================>] 765.70M   916KB/s    in 14m 34s 

2021-10-24 10:34:57 (897 KB/s) - ‘www.anatel.gov.br/dadosabertos/paineis_de_dados/acessos/acessos_telefonia_movel.zip’ saved [802894421/802894421]

FINISHED --2021-10-24 10:34:57--
Total wall clock time: 14m 35s
Downloaded: 1 files, 766M in 14m 34s (897 KB/s)


In [4]:
#salvar arquivos no Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import os
from zipfile import ZipFile
import zipfile

In [ ]:
with zipfile.ZipFile('/content/www.anatel.gov.br/dadosabertos/paineis_de_dados/acessos/acessos_telefonia_movel.zip') as z:
  print(*z.namelist(), sep='\n')

Acessos_Telefonia_Movel_2005-200901_Modalidade_Colunas.csv
Acessos_Telefonia_Movel_2005-200901_Tecnologia_Colunas.csv
Acessos_Telefonia_Movel_2005-2018_Modalidade.csv
Acessos_Telefonia_Movel_2005-2018_Tecnologia.csv
Acessos_Telefonia_Movel_200902-2018_Modalidade_Colunas.csv
Acessos_Telefonia_Movel_200902-2018_Tecnologia_Colunas.csv
Acessos_Telefonia_Movel_200903-2018_Modalidade_Colunas.csv
Acessos_Telefonia_Movel_200903-2018_Tecnologia_Colunas.csv
Acessos_Telefonia_Movel_201901-201906.csv
Acessos_Telefonia_Movel_201901-201906_Colunas.csv
Acessos_Telefonia_Movel_201907-201912.csv
Acessos_Telefonia_Movel_201907-201912_Colunas.csv
Acessos_Telefonia_Movel_202001-202006.csv
Acessos_Telefonia_Movel_202001-202006_Colunas.csv
Acessos_Telefonia_Movel_202007-202012.csv
Acessos_Telefonia_Movel_202007-202012_Colunas.csv
Acessos_Telefonia_Movel_2021.csv
Acessos_Telefonia_Movel_202101-202106_Colunas.csv
Acessos_Telefonia_Movel_202107-202112_Colunas.csv
Acessos_Telefonia_Movel_Pre_Pos_Total.csv
Acess

In [3]:
pasta = '/content/www.anatel.gov.br/dadosabertos/paineis_de_dados/acessos'
mobile = os.path.join(pasta, 'acessos_telefonia_movel.zip')

##### **Microdados (2019-2021)**

In [ ]:
#decodificando os arquivos (2019-2021)
  #Anatel separa por ano/semestre
with ZipFile(mobile) as z:
  with z.open(f'Acessos_Telefonia_Movel_201901-201906.csv') as f:
    tm_1 = pd.read_csv(f,sep=';', encoding='utf-8')

In [ ]:
with ZipFile(mobile) as z:
  with z.open(f'Acessos_Telefonia_Movel_201907-201912.csv') as f:
    tm_2 = pd.read_csv(f,sep=';', encoding='utf-8')

In [ ]:
with ZipFile(mobile) as z:
  with z.open(f'Acessos_Telefonia_Movel_202001-202006.csv') as f:
    tm_3 = pd.read_csv(f,sep=';', encoding='utf-8')

In [ ]:
with ZipFile(mobile) as z:
  with z.open(f'Acessos_Telefonia_Movel_202007-202012.csv') as f:
    tm_4 = pd.read_csv(f,sep=';', encoding='utf-8')

In [ ]:
with ZipFile(mobile) as z:
  with z.open(f'Acessos_Telefonia_Movel_2021.csv') as f:
    tm_5 = pd.read_csv(f,sep=';', encoding='utf-8')

In [ ]:
#renomeando as variáveis dos diferentes dataframes e excluindo variáveis desnecessárias
dfs = [tm_1, tm_2, tm_3, tm_4, tm_5]
ordem = ['ano', 'mes', 'sigla_uf', 'ddd', 'id_municipio', 'cnpj', 'empresa', 'porte_empresa', 'tecnologia', 
         'sinal', 'modalidade', 'pessoa', 'produto', 'acessos']
rename = {'Ano': 'ano','Mês':'mes', 'Grupo Econômico':'grupo_economico', 'Empresa':'empresa', 'CNPJ':'cnpj', 
          'Porte da Prestadora':'porte_empresa', 'UF':'sigla_uf', 'Município':'municipio', 'Código IBGE Município':'id_municipio', 
          'Código Nacional':'ddd', 'Código Nacional (Chip)':'ddd_chip', 'Modalidade de Cobrança':'modalidade', 'Tecnologia':'tecnologia', 
          'Tecnologia Geração':'sinal', 'Tipo de Pessoa':'pessoa', 'Tipo de Produto':'produto', 'Acessos':'acessos'}

for i in dfs:
    i.rename(columns=rename, inplace=True)
    i.drop(['grupo_economico', 'municipio', 'ddd_chip'], axis=1, inplace=True)
    i['acessos_total'] = i.groupby(['ano', 'mes', 'sigla_uf', 'id_municipio', 'ddd', 'cnpj', 'empresa', 'porte_empresa', 
                                    'tecnologia', 'sinal', 'modalidade', 'pessoa', 'produto'])['acessos'].transform(np.sum)
    i.sort_values(['ano', 'mes', 'sigla_uf', 'id_municipio', 'ddd', 'cnpj', 'empresa', 'porte_empresa', 'tecnologia', 
                   'sinal', 'modalidade', 'pessoa', 'produto'], inplace=True)
    i.drop_duplicates(subset=['ano', 'mes', 'sigla_uf', 'id_municipio', 'ddd', 'cnpj', 'empresa', 'porte_empresa', 
                                   'tecnologia', 'sinal', 'modalidade', 'pessoa', 'produto'], keep='first', inplace=True)
    i.drop('acessos', axis=1, inplace=True)
    i.rename(columns={'acessos_total':'acessos'}, inplace=True)
    i = i[ordem]

In [ ]:
#junção das bases
tm_mun = tm_1.append([tm_2, tm_3, tm_4, tm_5], ignore_index=True)
del tm_1, tm_2, tm_3, tm_4, tm_5

In [ ]:
#tipifcação da base
tm_mun['produto'] = tm_mun['produto'].str.lower()
tm_mun['id_municipio'] = tm_mun['id_municipio'].astype(str)
tm_mun['ddd'] = pd.to_numeric(tm_mun['ddd'], downcast='integer').astype(str)
tm_mun['cnpj'] = tm_mun['cnpj'].astype(str)
tm_mun = tm_mun[ordem]

In [ ]:
#tm_mun.to_csv('/content/telefonia_movel_municipio_2019-2021.csv', index=False) #salvar na memória do Colab
tm_mun.to_csv('/content/drive/MyDrive/br_anatel/telefonia_movel/output/telefonia_movel_municipio_2019-2021.csv', index=False, encoding="utf-8", na_rep="")

##### **Dados Tecnologia e Modalidade (2005-2018)**

In [ ]:
#decodificando os arquivos (2005-2018)
  #Anatel separa em tecnologia(tipo de conexão) e modalidade (tipo de planos: pré e pós pago)
with ZipFile(mobile) as z:
  with z.open(f'Acessos_Telefonia_Movel_2005-2018_Tecnologia.csv') as f:
   tm_tec = pd.read_csv(f,sep=';', encoding='utf-8')

In [ ]:
#renomeando/padronizando variaveis ref. tm_tec (telefonia móvel - tecnologia)
tm_tec.rename(columns={'Ano': 'ano','Mês':'mes', 'Grupo Econômico':'grupo_economico', 'Empresa':'empresa',
                 'CNPJ':'cnpj', 'Porte da Prestadora':'porte_empresa', 'UF':'sigla_uf', 'Região PGO':'outorga', 
                 'Código Nacional':'ddd', 'Tecnologia':'tecnologia', 'Tecnologia Geração':'sinal', 'Acessos':'acessos'}, inplace=True) 

In [ ]:
#exclusão de dados NaN (ddd & sigla_uf), início das observações completas em 2009
tm_tec.dropna(inplace=True)

In [ ]:
#exclusão de variáveis duplicadas
tm_tec['acessos_total'] = tm_tec.groupby(['ano', 'mes', 'ddd', 'cnpj', 'tecnologia', 'sinal'])['acessos'].transform(np.sum)
tm_tec.sort_values(['ano', 'mes', 'ddd', 'cnpj', 'tecnologia', 'sinal'], inplace=True)
tm_tec.drop_duplicates(subset=['ano', 'mes', 'ddd', 'cnpj', 'tecnologia', 'sinal'], keep='first', inplace=True)
tm_tec.drop('acessos', axis=1, inplace=True)
tm_tec.rename(columns={'acessos_total':'acessos'}, inplace=True)

In [ ]:
with ZipFile(mobile) as z:
  with z.open(f'Acessos_Telefonia_Movel_2005-2018_Modalidade.csv') as f:
   tm_mod = pd.read_csv(f,sep=';', encoding='utf-8')

In [ ]:
#renomeando/padronizando variaveis ref. tm_mod (telefonia_movel - modalidade)
tm_mod.rename(columns={'Ano': 'ano','Mês':'mes', 'Grupo Econômico':'grupo_economico', 'Empresa':'empresa',
                 'CNPJ':'cnpj', 'Porte da Prestadora':'porte_empresa', 'UF':'sigla_uf', 'Região PGO':'outorga', 
                 'Código Nacional':'ddd', 'Modalidade de Cobrança':'modalidade', 'Acessos':'acessos'}, inplace=True) 

In [ ]:
#exclusão de variáveis duplicadas
tm_mod['acessos_total'] = tm_mod.groupby(['ano', 'mes', 'ddd', 'cnpj', 'modalidade'])['acessos'].transform(np.sum)
tm_mod.sort_values(['ano', 'mes', 'ddd', 'cnpj', 'modalidade'], inplace=True)
tm_mod.drop_duplicates(subset=['ano', 'mes', 'ddd', 'cnpj', 'modalidade'], keep='first', inplace=True)
tm_mod.drop('acessos', axis=1, inplace=True)
tm_mod.rename(columns={'acessos_total':'acessos'}, inplace=True)

In [ ]:
#drop de colunas não necessárias
tm_tec.drop(['grupo_economico', 'outorga'], axis=1, inplace=True)
tm_mod.drop(['grupo_economico', 'outorga'], axis=1, inplace=True)

In [ ]:
#ordenação das colunas
tm_tec = tm_tec[['ano', 'mes', 'sigla_uf', 'ddd', 'cnpj', 'empresa',  'porte_empresa',  'tecnologia', 'sinal', 'acessos']]
tm_mod = tm_mod[['ano', 'mes', 'sigla_uf', 'ddd', 'cnpj', 'empresa',  'porte_empresa',  'modalidade', 'acessos']]

In [ ]:
#transformação da variável de int64 pra string
tm_tec['ddd'] = pd.to_numeric(tm_tec['ddd'], downcast='integer').astype(str)
tm_tec['cnpj'] = tm_tec['cnpj'].astype(str)

In [ ]:
#transformação da variável de int64 pra string
tm_mod['ddd'] = pd.to_numeric(tm_mod['ddd'], downcast='integer').astype(str)
tm_mod['cnpj'] = tm_mod['cnpj'].astype(str)

In [ ]:
tec = ['ano', 'mes', 'sigla_uf', 'ddd', 'cnpj', 'empresa', 'porte_empresa', 'tecnologia', 'sinal', 'acessos']
tm_mun_tec = tm_mun[tec]
tm_tec_atual = tm_tec.append(tm_mun_tec, ignore_index=True)

mod = ['ano', 'mes', 'sigla_uf', 'ddd', 'cnpj', 'empresa', 'porte_empresa', 'modalidade', 'acessos']
tm_mun_mod = tm_mun[mod]
tm_mod_atual = tm_mod.append(tm_mun_mod, ignore_index=True)

In [ ]:
#exportação arquivo csv na memória do Colab
#tm_tec.to_csv('/content/telefonia_movel_microdados_2009-2018_tecnologia.csv', index=False)
#tm_mod.to_csv('/content/telefonia_movel_microdados_2005-2018_modalidade.csv', index=False)
#ou salvar arquivos direto no Drive
tm_tec_atual.to_csv('/content/drive/MyDrive/br_anatel/telefonia_movel/output/telefonia_movel_microdados_tecnologia_2009-2021.csv', index=False)
tm_mod_atual.to_csv('/content/drive/MyDrive/br_anatel/telefonia_movel/output/telefonia_movel_microdados_modalidade_2005-2021.csv', index=False)

In [8]:
tm_mun = pd.read_csv('/content/drive/MyDrive/br_anatel/telefonia_movel/output/telefonia_movel_municipio_2019-2021.csv')

In [12]:
tm_mun.head(1)

,ano,mes,sigla_uf,ddd,id_municipio,cnpj,empresa,porte_empresa,tecnologia,sinal,modalidade,pessoa,produto,acessos
0,2019,1,AC,68,1200013,2421421000111,TIM,Grande Porte,GSM,2G,Pré-pago,Pessoa Física,voz+dados,585


In [14]:
#brasil
tm_br = tm_mod_atual.drop(['sigla_uf', 'ddd', 'empresa','cnpj', 'porte_empresa',  'modalidade'], axis = 1)
tm_br['acessos_total'] = tm_br.groupby(['ano', 'mes'])['acessos'].transform(np.sum)
tm_br.sort_values(['ano', 'mes'], inplace=True)
tm_br.drop_duplicates(subset=['ano', 'mes'], keep ='first', inplace = True)
tm_br.drop('acessos', axis=1, inplace=True)
tm_br.rename(columns={'acessos_total':'acessos'}, inplace=True)

#uf
tm_uf = tm_mod_atual.drop(['ddd', 'empresa','cnpj', 'porte_empresa',  'modalidade'], axis = 1)
tm_uf['acessos_total'] = tm_uf.groupby(['ano', 'mes', 'sigla_uf'])['acessos'].transform(np.sum)
tm_uf.sort_values(['ano', 'mes', 'sigla_uf'], inplace=True)
tm_uf.drop_duplicates(subset=['ano', 'mes', 'sigla_uf'], keep ='first', inplace = True)
tm_uf.drop('acessos', axis=1, inplace=True)
tm_uf.rename(columns={'acessos_total':'acessos'}, inplace=True)

#municipio
tm_mu = tm_mun.drop(['sigla_uf', 'ddd', 'empresa','cnpj', 'porte_empresa', 'tecnologia', 'sinal', 'modalidade', 'pessoa', 'produto'], axis = 1)
tm_mu['acessos_total'] = tm_mu.groupby(['ano', 'mes', 'id_municipio'])['acessos'].transform(np.sum)
tm_mu.sort_values(['ano', 'mes', 'id_municipio'], inplace=True)
tm_mu.drop_duplicates(subset=['ano', 'mes', 'id_municipio'], keep ='first', inplace = True)
tm_mu.drop('acessos', axis=1, inplace=True)
tm_mu.rename(columns={'acessos_total':'acessos'}, inplace=True)

In [16]:
with ZipFile(mobile) as z:
  with z.open(f'Densidade_Telefonia_Movel.csv') as f:
    densidade = pd.read_csv(f,sep=';', encoding='utf-8')

In [17]:
densidade.rename(columns={'Nível Geográfico Densidade':'Geografia'}, inplace=True)
densidade_br = densidade[densidade.Geografia.isin(['Brasil'])]
densidade_uf = densidade[densidade.Geografia.isin(['UF'])]
densidade_munic = densidade[densidade.Geografia.isin(['Municipio'])]

In [ ]:
densidade_br.drop(['UF', 'Município', 'Código IBGE', 'Geografia'], axis=1, inplace=True)
densidade_br.rename(columns={'Ano':'ano', 'Mês':'mes', 'Densidade':'densidade'}, inplace=True)
densidade_br['densidade'] = densidade_br['densidade'].apply(lambda x: str(x).replace(',','.'))
densidade_br = pd.merge(tm_br, densidade_br,
                         how='left', left_on=['ano', 'mes'],
                         right_on=['ano', 'mes'])
densidade_br['acessos'] = densidade_br['acessos'].astype('int64')
densidade_br.to_csv('/content/drive/MyDrive/br_anatel/telefonia_movel/output/densidade_telefonia_movel_brasil.csv', index=False, encoding="utf-8", na_rep="")

densidade_uf.drop(['Município', 'Código IBGE', 'Geografia'], axis=1, inplace=True)
densidade_uf.rename(columns={'Ano':'ano', 'Mês':'mes', 'UF':'sigla_uf', 'Densidade':'densidade'}, inplace=True)
densidade_uf['densidade'] = densidade_uf['densidade'].apply(lambda x: str(x).replace(',','.'))
densidade_uf = pd.merge(tm_uf, densidade_uf,
                         how='left', left_on=['ano', 'mes', 'sigla_uf'],
                         right_on=['ano', 'mes', 'sigla_uf'])
densidade_uf = densidade_uf.dropna()
densidade_uf = densidade_uf[densidade_uf.densidade != "0"]
densidade_uf['acessos'] = densidade_uf['acessos'].astype('int64')
densidade_uf.to_csv('/content/drive/MyDrive/br_anatel/telefonia_movel/output/densidade_telefonia_movel_uf.csv', index=False, encoding="utf-8", na_rep="")

densidade_munic.drop(['Município', 'UF', 'Geografia'], axis=1, inplace=True)
densidade_munic.rename(columns={'Ano':'ano', 'Mês':'mes', 'Código IBGE':'id_municipio', 'Densidade':'densidade'}, inplace=True)
densidade_munic['densidade'] = densidade_munic['densidade'].apply(lambda x: str(x).replace(',','.'))
densidade_munic['id_municipio'] = densidade_munic['id_municipio'].astype('int64')
densidade_munic = pd.merge(tm_mu, densidade_munic,
                         how='left', left_on=['ano', 'mes', 'id_municipio'],
                         right_on=['ano', 'mes', 'id_municipio'])
densidade_munic.to_csv('/content/drive/MyDrive/br_anatel/telefonia_movel/output/densidade_telefonia_movel_municipio.csv', index=False, encoding="utf-8", na_rep="")